## Este notebook apresenta a exploração dos dados após do pre-processamento

In [ ]:
import pandas as pd
import re
import gensim
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance

In [ ]:
#setting parameters for data visualization
np.set_printoptions(threshold=None, precision=2)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('precision', 2)

### Opening movie reviews

In [ ]:
messages = pd.read_csv('./chatdata/movie_lines_pre_processed_keras.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [ ]:
messages.columns = ['msg_line', 'user_id', 'movie_id', 'msg', 'msg_2', 'msg_pre_processed', 'target']

In [ ]:
messages.head(10)

### Print profile

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(messages, title="Pandas Profiling Report")

In [ ]:
profile

### Data exploration

In [ ]:
messages.info()

In [ ]:
messages.describe()

#### Data Analysis based on this article
- https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools?utm_source=medium&utm_medium=crosspost&utm_campaign=blog-exploratory-data-analysis-natural-language-processing-tools

In [ ]:
data = messages['msg_pre_processed']

### Number of characters of each message

In [ ]:
#histogram to display the number of character of each message
data.str.len().hist()

The number of characters are between 0 and 500

## Value types

In [ ]:
#checking the average word length
data_set = [type(item) for item in data]
data_set = set(data_set)
data_set

## Print float values

In [ ]:
float_val = [it for it in data if isinstance(it, float)]
print(len(float_val))
float_val = set(float_val)
print(float_val)

In [ ]:
messages[messages['msg_pre_processed'].isna()]

### Removing non-alphabetical messages

In [ ]:
#example of sequence of null messages
#messages[538:540]

In [ ]:
#filling the nan messages with a string
messages = messages.fillna('UNKNOWN')
data = messages['msg']

### Number of words for each message

In [ ]:
#number of words for each message
data.str.split(' ').\
    map(lambda x: len(str(x))).\
    hist()

The number of words are between 0 and 100

### Average word length

In [ ]:
#checking the average word length
data.str.split(' ').\
   apply(lambda x : [len(i) for i in x]). \
   map(lambda x: np.mean(x)).hist()

The length of words goes from 0 to 15

### Distribuition of stopwords in the mesages

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
#checkig the distribuition of stopwrds in the mesages
stop=set(stopwords.words('english'))
corpus=[]
msg_ = data.str.split()
msg_ = msg_.values.tolist()
corpus=[word for i in msg_ for word in i]

from collections import defaultdict
dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1

In [ ]:
len(list(dic.values()))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = dic.keys()
y = dic.values()
plt.subplots(figsize=(30,5))
plt.bar(x, y, 1, color='b')
plt.xticks(list(x), rotation=90, fontsize=14)
plt.show()

The majority of stop words are 'to', 'you', 'the' and 'a'

### Occurences of each word

In [ ]:
from collections import Counter
import seaborn as sns

In [ ]:
#ploting the occurences of each word
counter=Counter(corpus)
most=counter.most_common()

x, y= [], []
for word,count in most[:40]:
    if (word not in stop):
        x.append(word)
        y.append(count)
        
sns.barplot(x=y,y=x)

The word 'I' has the biggest occurrence. There are a lot of messages like dashes that can be removed

### Get the most frequent n-grams

In [ ]:
from nltk.util import ngrams

In [ ]:
#get the most frequent n-grams
def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:10]

In [ ]:
def plot_n_gram(corpus, n=2):
    top_n_bigrams=get_top_ngram(corpus,n)[:10]
    x,y=map(list,zip(*top_n_bigrams))
    sns.barplot(x=y,y=x)

In [ ]:
plot_n_gram(data, 2)

The more frequent bigram is 'you re'

In [ ]:
plot_n_gram(data, 3)

There are a lot of short questions like 'what do you...?', 'what are you...?', and answers like 'you want to...'

In [ ]:
plot_n_gram(data, 4)

There are a lot of repeated questions and answers or part of them

### Sentimental analysis

In [ ]:
from textblob import TextBlob

In [ ]:
#checkig the polarity of the messages
def polarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
messages['polarity_score']=data.\
   apply(lambda x : polarity(x))
messages['polarity_score'].hist()

The majority of polarity is neutral

### Named entity recognition

- TYPE	DESCRIPTION
- PERSON	People, including fictional.
- NORP	Nationalities or religious or political groups.
- FAC	Buildings, airports, highways, bridges, etc.
- ORG	Companies, agencies, institutions, etc.
- GPE	Countries, cities, states.
- LOC	Non-GPE locations, mountain ranges, bodies of water.
- PRODUCT	Objects, vehicles, foods, etc. (Not services.)
- EVENT	Named hurricanes, battles, wars, sports events, etc.
- WORK_OF_ART	Titles of books, songs, etc.
- LAW	Named documents made into laws.
- LANGUAGE	Any named language.
- DATE	Absolute or relative dates or periods.
- TIME	Times smaller than a day.
- PERCENT	Percentage, including ”%“.
- MONEY	Monetary values, including unit.
- QUANTITY	Measurements, as of weight or distance.
- ORDINAL	“first”, “second”, etc.
- CARDINAL	Numerals that do not fall under another type.

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
#transform the messages in a big document
def get_entities(corpus, int_limit=0, end_limit=100):
    msg_all = ' '.join(data[int_limit:end_limit])
    doc = nlp(msg_all)

    entity_list = [(x.text,x.label_) for x in doc.ents]
    
    return entity_list

In [ ]:
%%time

ent_list = set()
skip = 1000
i = 0
lim = len(data)
#lim = 20000

while(i < lim):    
    ent_list = set(get_entities(data, i, i+skip))
    ent_list.union(ent_list)
    i = i + skip
    print('Processed '+str(i)+' of '+str(lim)+' messages')

In [ ]:
len(ent_list)

In [ ]:
ent_list

There are some personal names, real locals, name of organizations and work of art that can be removed or replaced for generic terms. 

In [ ]:
ent_df = pd.DataFrame(list(ent_list))

In [ ]:
ent_df

### Entity frequency

In [ ]:
def ner(text):
    doc=nlp(text)
    return [X.label_ for X in doc.ents]

In [ ]:
%%time
#count the number of each entity
ent=data.\
    apply(lambda x : ner(x))
ent=[x for sub in ent for x in sub]

counter=Counter(ent)
count=counter.most_common()

In [ ]:
x,y=map(list,zip(*count))
sns.barplot(x=y,y=x)

### Most comon tokens per entity

In [ ]:
def ner(text,ent="PERSON"):
    doc=nlp(text)
    return [X.text for X in doc.ents if X.label_ == ent]

In [ ]:
%%time
#count the tokens for a given entity type
gpe=data.apply(lambda x: ner(x, 'PERSON'))
gpe=[i for x in gpe for i in x]
counter=Counter(gpe)

In [ ]:
x,y=map(list,zip(*counter.most_common(10)))
sns.barplot(y,x)

## Parts of Speach Tagging

There are eight main parts of speech:

- Noun (NN)- Joseph, London, table, cat, teacher, pen, city
- Verb (VB)- read, speak, run, eat, play, live, walk, have, like, are, is
- Adjective(JJ)- beautiful, happy, sad, young, fun, three
- Adverb(RB)- slowly, quietly, very, always, never, too, well, tomorrow
- Preposition (IN)- at, on, in, from, with, near, between, about, under
- Conjunction (CC)- and, or, but, because, so, yet, unless, since, if
- Pronoun(PRP)- I, you, we, they, he, she, it, me, us, them, him, her, this
- Interjection (INT)- Ouch! Wow! Great! Help! Oh! Hey! Hi!

In [ ]:
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize

In [ ]:
def remove_duplicated_spaces(corpus):
    #remove duplicated spaces
    corpus_alt = re.sub(r' +', ' ', corpus)

    return corpus_alt

In [ ]:
msg_no_space = [remove_duplicated_spaces(str(m)) for m in data]
msg_no_space = pd.Series(msg_no_space)

In [ ]:
def pos(text):
    #matched = re.match(r" +", text)
    if text != ' ':
        #print(text)
        pos=nltk.pos_tag(word_tokenize(text))
        #print(pos)
        pos=list(map(list,zip(*pos)))[1]
        return pos
    return

In [ ]:
%%time
tags=msg_no_space[0:5000].apply(lambda x : pos(x))
#tags = [pos(m) for m in msg_no_space[0:10000]]
#tags
tags=[x for l in tags for x in l]
counter=Counter(tags)

In [ ]:
x,y=list(map(list,zip(*counter.most_common(7))))
sns.barplot(x=y,y=x)

In [ ]:
### most commonly nouns
def get_adjs(text, _tag='PRP'):
    adj=[]
    pos=nltk.pos_tag(word_tokenize(text))
    for word,tag in pos:
        if tag == _tag:
            adj.append(word)
    return adj

In [ ]:
%%time
words=data.apply(lambda x : get_adjs(x))
words=[x for l in words for x in l]
counter=Counter(words)

In [ ]:
x,y=list(map(list,zip(*counter.most_common(7))))
sns.barplot(x=y,y=x)

## Text complexy

Flesch Reading Ease (FRE)
<br>Higher scores indicate material that is easier to read, lower numbers mark harder-to-read passages:
- 0-30 College
- 50-60 High school
- 60+ Fourth grade

In [ ]:
from textstat import flesch_reading_ease

In [ ]:
%%time
reading = data.\
   apply(lambda x : flesch_reading_ease(x))
reading.hist()

The text is easy to read

In [ ]:
x=[i for i in range(len(reading)) if reading[i]<0]

messages.iloc[x]['msg_pre_processed'].head()